### SocialAL Models
# Data simulation using best-fit parameters - multiple subjects
KLS 8.30.19; update 7.7.22; update 10.31.22
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.11.0 (v3.11.0:deaf509e8f, Oct 24 2022, 14:43:23) [Clang 13.0.0 (clang-1300.0.29.30)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import beta
from scipy.stats import gamma
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [3]:
run ../common_functions.ipynb

In [4]:
run ../single_alpha_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    
    alpha = params[0]
    beta = params[1]

    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    param1=[]
    param2=[]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    trial = []
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Trial
        trial.append(x+1)
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # after choice, update probability
        if choice != 1: 
            Probs[t] = update_prob(response, Probs[t], alpha)
        # then update value
        EVs[t] = update_value(Probs[t]) 

    data = {'Trial': trial, 'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses, 
            'Alpha': [alpha]* tn * 3, 'Beta': [beta] * tn * 3}    
    return data

In [8]:
#sim_data(15,[.2,2])

### Pull in best-fit parameters from participants

In [9]:
dt = pd.read_csv('../../../output/single_alpha_model_params.csv')
print(dt)

          id     alpha          beta      X.LLH
0   sub-2013  0.003099  2.000000e+01  47.030085
1   sub-2007  0.212075  3.359905e-01  51.644795
2   sub-2006  0.000000  1.000000e-10  60.996952
3   sub-2012  0.000000  1.000000e-10  62.383246
4   sub-2004  0.180936  1.098443e+00  35.077751
..       ...       ...           ...        ...
58  sub-2009  0.000000  1.000000e-10  62.383246
59  sub-2037  1.000000  2.120724e-01  50.237647
60  sub-2023  0.482881  6.157203e-01  34.813597
61  sub-2022  0.000000  1.000000e-10  60.996952
62  sub-2036  0.000000  1.000000e-10  62.383246

[63 rows x 4 columns]


In [10]:
alphas = dt['alpha']
betas = dt['beta']
parent_sub = dt['id']
print(alphas)

0     0.003099
1     0.212075
2     0.000000
3     0.000000
4     0.180936
        ...   
58    0.000000
59    1.000000
60    0.482881
61    0.000000
62    0.000000
Name: alpha, Length: 63, dtype: float64


### Simulate data

In [11]:
data = pd.DataFrame(columns= ['Trial', 'Stim_Sequence', 'Choice', 'Trustee_Response', 'Alpha', 'Beta', 'Subject'])
for p in range(len(alphas)):
    for q in range(1,11):
        dt = sim_data(15, [alphas[p], betas[p]])
        dt['Subject'] = parent_sub[p] + '_' + str(q) #[p + 1] * 45
        dt = pd.DataFrame(dt)
        data = pd.concat([data, dt])
    
data.to_csv(path_or_buf = '../../../output/simulation/part_params/sim_1alpha_model_data.csv', index = False)

In [12]:
print(data)

   Trial Stim_Sequence Choice Trustee_Response     Alpha          Beta  \
0      1             1      1                1  0.003099  2.000000e+01   
1      2             2      1                0  0.003099  2.000000e+01   
2      3             2      1                0  0.003099  2.000000e+01   
3      4             1      2                1  0.003099  2.000000e+01   
4      5             1      1                0  0.003099  2.000000e+01   
..   ...           ...    ...              ...       ...           ...   
40    41             0      2                0  0.000000  1.000000e-10   
41    42             1      1                0  0.000000  1.000000e-10   
42    43             1      4                0  0.000000  1.000000e-10   
43    44             0      4                1  0.000000  1.000000e-10   
44    45             2      1                0  0.000000  1.000000e-10   

        Subject  
0    sub-2013_1  
1    sub-2013_1  
2    sub-2013_1  
3    sub-2013_1  
4    sub-2013_1  
.. 